## Dependency parsing

In [2]:
import spacy

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
doc = nlp("The quick brown fox jumps over the lazy dog.")

In [5]:
for token in doc:
    print(f" {token.text} -> {token.dep_} -> {token.head.text}")

 The -> det -> fox
 quick -> amod -> fox
 brown -> amod -> fox
 fox -> nsubj -> jumps
 jumps -> ROOT -> jumps
 over -> prep -> jumps
 the -> det -> dog
 lazy -> amod -> dog
 dog -> pobj -> over
 . -> punct -> jumps


In [ ]:
'''
                   jumps
               /    |      \
            fox  	over     .
          / | \     	\
      The quick brown   dog
                        /  \
                      the  lazy
'''

In [ ]:
"""
	Label	Meaning
	nsubj	Nominal subject
	dobj	Direct object
	pobj	Object of a preposition
	amod	Adjectival modifier
	prep	Prepositional modifier
	ROOT	Main verb/root of sentence
	det		Determiner
	advmod	Adverbial modifier
"""

In [8]:
def extract_subject_verb_object(text):
    doc = nlp(text)
    
    subject = ""
    verb = ""
    object = ""
    
    for token in doc:
        if token.dep_ == "nsubj":
            subject = token.text
        if token.dep_ == "ROOT":
            verb = token.text
        if token.dep_ == "dobj" or token.dep_ == "pobj":
            object = token.text
            
    return subject, verb, object

In [9]:
if __name__ == "__main__":
    print("Dependency Parser")
    sentence = input("Enter a sentence: ")
    subject, verb, obj = extract_subject_verb_object(sentence)

    print(f"\nExtracted parts:")
    print(f"Subject: {subject}")
    print(f"Verb: {verb}")
    print(f"Object: {obj}")


Dependency Parser

Extracted parts:
Subject: cat
Verb: fighting
Object: mouse


## Chunking

In [2]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.chunk import RegexpParser

In [3]:
sentence = "The quick brown fox jumps over the lazy dog."

In [4]:
tokens = word_tokenize(sentence)
pos_tag = pos_tag(tokens)
print(pos_tag)

[('The', 'DT'), ('quick', 'JJ'), ('brown', 'NN'), ('fox', 'NN'), ('jumps', 'VBZ'), ('over', 'IN'), ('the', 'DT'), ('lazy', 'JJ'), ('dog', 'NN'), ('.', '.')]


In [ ]:
""" 
Noun (NN)
Verb (V)
Determiner (DT)
Adjective (JJ)
Adverb (ADV)
Preposition (PREP)
Conjunction (CONJ)


Symbol	Meaning
	?	Optional (0 or 1 occurrence)
	*	Zero or more occurrences
	+	One or more occurrences


"The quick fox" → DT + JJ + NN 
"lazy dog" → JJ + NN 
"dog" → NN 
"""

In [ ]:
chunk_grammar = """ NP: {<DT>?<JJ>*<NN>}
					VP: {<V><NP>}
					PP: {<IN><NP>}
				"""
    
# NP : Noun Phrase, VP : Verb Phrase, PP : Preposition Phrase

chunk_parser = RegexpParser(chunk_grammar)
chunk_tree = chunk_parser.parse(pos_tag)

In [6]:
chunk_tree.draw()

In [7]:
chunk_tree.leaves()


[('The', 'DT'),
 ('quick', 'JJ'),
 ('brown', 'NN'),
 ('fox', 'NN'),
 ('jumps', 'VBZ'),
 ('over', 'IN'),
 ('the', 'DT'),
 ('lazy', 'JJ'),
 ('dog', 'NN'),
 ('.', '.')]

In [10]:
chunk_tree.height()

4

In [11]:
chunk_tree.pretty_print()

                                 S                                             
     ____________________________|____________________________                  
    |      |            |                |                    PP               
    |      |            |                |        ____________|_____            
    |      |            NP               NP      |                  NP         
    |      |     _______|________        |       |       ___________|______     
jumps/VBZ ./. The/DT quick/JJ brown/NN fox/NN over/IN the/DT     lazy/JJ dog/NN



In [12]:
import nltk

# Define grammar
grammar = nltk.CFG.fromstring("""
  S -> NP VP
  NP -> DT NN
  VP -> V NP
  DT -> 'the'
  NN -> 'dog' | 'cat'
  V -> 'chased'
""")

parser = nltk.ChartParser(grammar)
sentence = ['the', 'dog', 'chased', 'the', 'cat']

for tree in parser.parse(sentence):
    tree.pretty_print()


              S               
      ________|_____           
     |              VP        
     |         _____|___       
     NP       |         NP    
  ___|___     |      ___|___   
 DT      NN   V     DT      NN
 |       |    |     |       |  
the     dog chased the     cat

